In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image
import ast
sys.path.append('../input/weightedboxesfusion/Weighted-Boxes-Fusion')

# 1280
# x1.25 = 1600
# x1.5  = 1920
# x1.75 = 2240
# x2    = 2560
# x2.25 = 2880
# x2.5  = 3200
# x3    = 3840
# x3.25 = 4160
# x3.5  = 4480
# x3.75 = 4800


Best_Models = ['../input/m6-2560-20ep-weights/fold0.pt',
               '../input/m6-2560-20ep-randomresize-weights/fold1.pt',
               '../input/m6-2560-20ep-weights/fold2.pt',
               '../input/m6-2560-20ep-randomresize-weights/fold3.pt',
               '../input/m6-2560-20ep-randomresize-weights/fold4.pt'] + \
               glob.glob('../input/m6-2880-15epfinetune/*pt')
#glob.glob('../input/m6-2560-20ep-video-weights/*pt') + \

print(Best_Models)

# best params in CV
# Input:2560
# TTA:True
# NMS-IoU:0.35
# Expansion:-0.04

#IMG_SIZES=[3840]*len(Best_Models)
IMG_SIZES=[2560]*5 + [2880]*5

#TTAs=[False]*len(Best_Models)
TTAs=[True]*5 + [True]*5

IOU=0.4
WBF_IOU=0.475
ENSEMBLE_METHOD='WBF'#NMS,NMW,WBF

# use custom tta
REPO_DIR_PATH = '../input/custom-tta/yolo_repo_vh'# hflip TTA

BOX_EXPANSION=-0.04

CONFS = [0.4]*len(Best_Models)
#CONFS = [0.134, 0.156, 0.234, 0.235, 0.16]
FINAL_CONF = 0#0.4#0.4

TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef'

In [ ]:
assert ENSEMBLE_METHOD in ["WBF", "NMW", "NMS"]

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def coco2voc(bboxes, image_height=720, image_width=1280):
    bboxes  = coco2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2voc(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row


np.random.seed(8)
colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
colors=(255,0,0)

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolo-arial/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def load_model(Best_Model, conf=0.25, iou=0.50):
    model = torch.hub.load(REPO_DIR_PATH,
                           'custom',
                           path=Best_Model,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model


def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
def expand_bboxes(bboxes, expansion=0.1):
    if len(bboxes) == 0:
        return bboxes
    else:
        new_bboxes = []
        for bbox in bboxes:
            xmin, ymin, w, h = bbox
            delta_w = w*expansion*0.5
            delta_h = h*expansion*0.5
            
            new_xmin = int(round(np.clip(xmin - delta_w, 0, 1280)))
            new_ymin = int(round(np.clip(ymin - delta_h, 0, 720)))
            new_w = int(round(w + 2.0*delta_w))
            new_h = int(round(h + 2.0*delta_h))
            
            new_bboxes.append([new_xmin, new_ymin, new_w, new_h])
            
        return np.array(new_bboxes)
    
def filter_bboxes(bboxes, confs, conf_thresh):
    if len(confs) == 0:
        return [], []
    else:
        filtered_bboxes, filtered_confs = [], []
        for bbox, conf in zip(bboxes, confs):
            if conf > conf_thresh:
                filtered_bboxes.append(bbox)
                filtered_confs.append(conf)
        return filtered_bboxes, filtered_confs

In [ ]:
from ensemble_boxes import *

def run_wbf(bboxes, confs, image_size=1280, iou_thr=0.5, skip_box_thr=0.001, weights=None, method='WBF'):
    num_ensemble = len(bboxes)
    boxes =  [bbox/(image_size) for bbox in bboxes]
    
    scores = [conf for conf in confs]
    labels = [np.ones(conf.shape[0]) for conf in confs]
    
    if method == 'WBF':
        boxes, scores, labels = weighted_boxes_fusion(boxes,
                                                      scores,
                                                      labels,
                                                      weights=[1]*num_ensemble,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
    elif method == 'NMW':
        boxes, scores, labels = non_maximum_weighted(boxes,
                                                     scores,
                                                     labels,
                                                     weights=[1]*num_ensemble,
                                                     iou_thr=iou_thr,
                                                     skip_box_thr=skip_box_thr)
    elif method == 'NMS':
        boxes, scores, labels = nms(boxes,
                                    scores,
                                    labels,
                                    weights=[1]*num_ensemble,
                                    iou_thr=iou_thr)
    else:
        0/0
        
    
    boxes = boxes*(image_size-1)
    boxes=voc2coco(boxes).astype(int)
    
    return boxes, scores, labels

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
models = []
for Best_Model, CONF in zip(Best_Models, CONFS):
    models.append(load_model(Best_Model, conf=CONF, iou=IOU))

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    bboxes_all = []
    confs_all = []
    for model, IMG_SIZE, TTA in zip(models, IMG_SIZES, TTAs):
        bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=TTA)
        
        if len(bboxes) > 0:
            bboxes_all.append(coco2voc(bboxes).astype(int))
            confs_all.append(confs)
        
    if len(bboxes_all) > 1:
        bboxes_wbf, confs_wbf, _ = run_wbf(bboxes_all,
                                           confs_all,
                                           image_size=1280,
                                           iou_thr=WBF_IOU,
                                           method=ENSEMBLE_METHOD)
        if FINAL_CONF != 0:
            bboxes_wbf, confs_wbf = filter_bboxes(bboxes_wbf, confs_wbf, FINAL_CONF)

    else:
        bboxes_wbf = []
        confs_wbf = []
    
    if BOX_EXPANSION != 0:
        bboxes_wbf = expand_bboxes(bboxes_wbf, BOX_EXPANSION)
        
    annot = format_prediction(bboxes_wbf, confs_wbf)
    pred_df['annotations'] = annot
    env.predict(pred_df)
    
    if idx<3:
        display(show_img(img, bboxes_wbf, bbox_format='coco'))

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()